# Orchestrate Jobs to Train and Evaluate Models with Amazon SageMaker Pipelines

Amazon SageMaker 파이프라인은 기계 학습(ML) 애플리케이션 개발자와 운영 엔지니어에게 SageMaker 작업을 조정하고 재현 가능한 ML 파이프라인을 작성할 수 있는 기능을 제공합니다. 또한 짧은 대기 시간으로 실시간으로 추론을 위해 맞춤형 모델을 배포하고, Batch Transform으로 오프라인 추론을 실행하고, 아티팩트 계보를 추적할 수 있습니다. 그들은 프로덕션 워크플로 배포 및 모니터링, 모델 아티팩트 배포, 간단한 인터페이스를 통해 아티팩트 계보 추적에서 건전한 운영 관행을 제정하고 ML 애플리케이션 개발을 위한 안전 및 모범 사례 패러다임을 준수할 수 있습니다.

SageMaker 파이프라인 서비스는 선언적 JSON 사양인 SageMaker 파이프라인 DSL(도메인 특정 언어)을 지원합니다. 이 DSL은 파이프라인 매개변수 및 SageMaker 작업 단계의 DAG(방향성 비순환 그래프)를 정의합니다. SageMaker Python SDK(소프트웨어 개발자 키트)는 엔지니어와 과학자에게 이미 익숙한 구성을 사용하여 파이프라인 DSL 생성을 간소화합니다.

## Runtime

This notebook takes approximately an hour to run.

## Contents

1. [SageMaker Pipelines](#SageMaker-Pipelines)
1. [Notebook Overview](#Notebook-Overview)
1. [A SageMaker Pipeline](#A-SageMaker-Pipeline)
1. [Dataset](#Dataset)
1. [Define Parameters to Parametrize Pipeline Execution](#Define-Parameters-to-Parametrize-Pipeline-Execution)
1. [Define a Cache](#Define-a-Cache)
1. [Define a Processing Step for Feature Engineering](#Define-a-Processing-Step-for-Feature-Engineering)
1. [Define a Training Step to Train a Model](#Define-a-Training-Step-to-Train-a-Model)
1. [Define a Model Evaluation Step to Evaluate the Trained Model](#Define-a-Model-Evaluation-Step-to-Evaluate-the-Trained-Model)
1. [Define a Create Model Step to Create a Model](#Define-a-Create-Model-Step-to-Create-a-Model)
1. [Define a Transform Step to Perform Batch Transformation](#Define-a-Transform-Step-to-Perform-Batch-Transformation)
1. [Define a Register Model Step to Create a Model Package](#Define-a-Register-Model-Step-to-Create-a-Model-Package)
1. [Define a Fail Step to Terminate the Pipeline Execution and Mark it as Failed](#Define-a-Fail-Step-to-Terminate-the-Pipeline-Execution-and-Mark-it-as-Failed)
1. [Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry, Or Terminate the Execution in Failed State](#Define-a-Condition-Step-to-Check-Accuracy-and-Conditionally-Create-a-Model-and-Run-a-Batch-Transformation-and-Register-a-Model-in-the-Model-Registry,-Or-Terminate-the-Execution-in-Failed-State)
1. [Define a Pipeline of Parameters, Steps, and Conditions](#Define-a-Pipeline-of-Parameters,-Steps,-and-Conditions)
1. [Submit the pipeline to SageMaker and start execution](#Submit-the-pipeline-to-SageMaker-and-start-execution)
1. [Pipeline Operations: Examining and Waiting for Pipeline Execution](#Pipeline-Operations:-Examining-and-Waiting-for-Pipeline-Execution)
    1. [Examining the Evaluation](#Examining-the-Evaluation)
    1. [Lineage](#Lineage)
    1. [Parametrized Executions](#Parametrized-Executions)

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - SageMaker 작업 및 리소스 생성을 조정하기 위한 단계 및 조건의 DAG입니다.
* Processing job steps - feature 엔지니어링, 데이터 유효성 검사, 모델 평가 및 모델 해석과 같은 데이터 처리 워크로드를 실행하기 위한 SageMaker의 간소화된 관리 환경입니다.
* Training job steps - 훈련 데이터 세트의 예를 제시하여 모델이 예측을 하도록 가르치는 반복적인 프로세스입니다.
* Conditional execution steps - 파이프라인에서 분기의 조건부 실행을 제공하는 단계입니다.
* Register model steps - Amazon SageMaker에서 배포 가능한 모델을 생성하는 데 사용할 수 있는 모델 레지스트리에서 모델 패키지 리소스를 생성하는 단계입니다.
* Create model steps - 변환 단계에서 사용하기 위한 모델을 작성하거나 나중에 엔드포인트로 발행하는 단계입니다.
* Transform job steps - 데이터 세트에서 훈련 또는 추론을 방해하는 노이즈 또는 편향을 제거하고, 대규모 데이터 세트에서 추론을 가져오고, 영구 끝점이 필요하지 않을 때 추론을 실행하기 위해 데이터 세트를 사전 처리하는 일괄 변환입니다.
* Fail steps - 파이프라인 실행을 중지하고 파이프라인 실행을 실패로 표시하는 단계입니다.
* Parametrized Pipeline executions - 지정된 매개변수에 따라 파이프라인 실행의 변형을 활성화합니다.

## Notebook Overview

This notebook shows how to:

* SageMaker 파이프라인을 매개변수화하는 데 사용할 수 있는 파이프라인 매개변수 세트를 정의합니다.
* clearning, feature 엔지니어링 및 입력 데이터를 학습 및 테스트 데이터 세트로 분할하는 처리 단계를 정의합니다.
* 전처리된 훈련 데이터 세트에서 모델을 훈련하는 훈련 단계를 정의합니다.
* 테스트 데이터 세트에서 훈련된 모델의 성능을 평가하는 처리 단계를 정의합니다.
* 훈련에 사용된 모델 아티팩트에서 모델을 생성하는 모델 생성 단계를 정의합니다.
* 생성된 모델을 기반으로 일괄 변환을 수행하는 변환 단계를 정의합니다.
* 모델을 훈련하는 데 사용되는 추정기 및 모델 아티팩트에서 모델 패키지를 생성하는 모델 등록 단계를 정의합니다.
* 이전 단계의 출력을 기반으로 조건을 측정하고 다른 단계를 조건부로 실행하는 조건부 단계를 정의합니다.
* 실행 실패의 원인을 나타내는 사용자 정의 오류 메시지로 실패 단계를 정의하십시오.
* 정의된 매개변수 및 단계를 사용하여 DAG에서 파이프라인 정의를 정의하고 생성합니다.
* 파이프라인 실행을 시작하고 실행이 완료될 때까지 기다립니다.
* 검토를 위해 S3 버킷에서 모델 평가 보고서를 다운로드합니다.
* 두 번째 파이프라인 실행을 시작합니다.

## A SageMaker Pipeline

생성하는 파이프라인은 사전 처리, 교육, 평가, 모델 생성, 일괄 변환 및 모델 등록의 일반적인 기계 학습(ML) 애플리케이션 패턴을 따릅니다:

![A typical ML Application pipeline](img/pipeline-full.png)

## Dataset

사용하는 데이터세트는 [UCI Machine Learning 전복 데이터세트]입니다.(https://archive.ics.uci.edu/ml/datasets/abalone) [1].  이 작업의 목적은 물리적 측정에서 전복 달팽이의 나이를 결정하는 것입니다. 핵심적으로 이것은 회귀 문제입니다.

데이터 세트에는 길이(가장 긴 껍질 측정), 직경(길이에 수직인 직경), 높이(껍질에 고기가 있는 높이), whole_weight(전체 전복의 무게), shucked_weight(고기 무게)와 같은 여러 기능이 포함되어 있습니다. , viscera_weight(출혈 후 내장 무게), shell_weight(건조 후 무게), 성별('M', 'F', 'I'에서 'I'는 유아임) 및 ring(정수).

링의 수는 나이에 대한 좋은 근사값으로 판명되었습니다(나이는 링 + 1.5). 그러나 이 숫자를 얻으려면 원뿔을 통해 껍질을 자르고 단면을 염색하고 현미경으로 고리의 수를 세는 작업이 필요하며 이는 시간이 많이 걸리는 작업입니다. 그러나 다른 물리적 측정은 더 쉽게 결정할 수 있습니다. 데이터세트를 사용하여 이러한 다른 물리적 측정을 통해 가변 링의 예측 모델을 구축합니다.

데이터를 S3 버킷에 업로드하기 전에 [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/)를 최신 버전으로 업그레이드하고 나중에 이 노트북에서 사용할 수 있는 몇 가지 상수를 수집하십시오.

> [1] Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.

In [1]:
import boto3
import sagemaker

# 여러명 동시 작업을 위해 다음 변수 커스터마이징
pipeline_name = f"NickName-Pipeline" # 다음에서 실습할 파이프라인명을 변경해주세요.
model_package_group_name = f"NickName-PackageGroupName" # 모델 레지스트리에 등록될 package group name을 변경해주세요.

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

이제 기본 버킷에 데이터를 업로드합니다. 'input_data_uri'에 대한 자체 데이터 세트 경로를 입력할 수도 있습니다.

In [3]:
!mkdir -p data

In [4]:
local_path = "data/abalone-dataset.csv"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-sample-files").download_file(
    "datasets/tabular/uci_abalone/abalone.csv", local_path
)

base_uri = f"s3://{default_bucket}/abalone"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-ap-northeast-2-387402383014/abalone/abalone-dataset.csv


모델 생성 후 일괄 변환을 위한 두 번째 데이터 세트를 다운로드합니다. 적절하게 `batch_data_uri`에 대한 자체 데이터 세트 경로를 입력할 수도 있습니다.

In [5]:
local_path = "data/abalone-dataset-batch"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
    "dataset/abalone-dataset-batch", local_path
)

base_uri = f"s3://{default_bucket}/abalone"
batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

s3://sagemaker-ap-northeast-2-387402383014/abalone/abalone-dataset-batch


## Define Parameters to Parametrize Pipeline Execution

파이프라인을 매개변수화하는데 사용할 수 있는 파이프라인 매개변수를 정의합니다. 매개변수를 사용하면 파이프라인 정의를 수정하지 않고도 사용자 지정 파이프라인 실행 및 일정을 사용할 수 있습니다.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

이러한 매개변수는 파이프라인 실행 시 재정의할 수 있는 기본값 제공을 지원합니다. 지정된 기본값은 매개변수 유형과 동일한 타입이어야 합니다.

The parameters defined in this workflow include:

* `processing_instance_type` - 처리 작업의 `ml.*` 인스턴스 유형.
* `processing_instance_count` - 처리 작업의 인스턴스 수입니다.
* `instance_type` - 학습 작업의 `ml.*` 인스턴스 유형입니다.
* `model_approval_status` - CI/CD 목적으로 훈련된 모델에 등록하기 위한 승인 상태입니다(기본값은 "PendingManualApproval").
* `input_data` - 입력 데이터의 S3 버킷 URI 위치입니다.
* `batch_data` - 배치 데이터의 S3 버킷 URI 위치입니다.
* `mse_threshold` - 모델의 정확도를 확인하는 데 사용되는 MSE(평균 제곱 오차) 임계값입니다.

In [6]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)


processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.large"
)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.large")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

## Define a Cache
- 참고: 캐싱 파이프라인 단계:  [Caching Pipeline Steps](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines-caching.html)

In [31]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, 
                           expire_after="7d")

![Define Parameters](img/pipeline-1.png)

## Define a Processing Step for Feature Engineering

먼저 처리 단계에서 지정한 전처리 스크립트를 개발합니다.

이 노트북 셀은 전처리 스크립트가 포함된 `preprocessing_abalone.py` 파일을 작성합니다. 스크립트를 업데이트하고 이 셀을 다시 실행하여 덮어쓸 수 있습니다. 전처리 스크립트는 `scikit-learn`을 사용하여 다음을 수행합니다.

* 누락된 성별 카테고리 데이터를 채우고 훈련에 적합하도록 인코딩합니다.
* 성별 및 고리 숫자 데이터를 제외한 모든 숫자 필드를 확장하고 정규화합니다.
* 데이터를 훈련, 검증 및 테스트 데이터 세트로 분할합니다.

처리 단계는 입력 데이터에 대한 스크립트를 실행합니다. 교육 단계에서는 사전 처리된 교육 기능과 레이블을 사용하여 모델을 교육합니다. 평가 단계에서는 훈련된 모델과 사전 처리된 테스트 기능 및 레이블을 사용하여 모델을 평가합니다.

In [32]:
!mkdir -p abalone

In [33]:
%%writefile abalone/preprocessing.py
import argparse
import os
import requests
import tempfile

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Since we get a headerless CSV file, we specify the column names here.
feature_columns_names = [
    "sex",
    "length",
    "diameter",
    "height",
    "whole_weight",
    "shucked_weight",
    "viscera_weight",
    "shell_weight",
]
label_column = "rings"

feature_columns_dtype = {
    "sex": str,
    "length": np.float64,
    "diameter": np.float64,
    "height": np.float64,
    "whole_weight": np.float64,
    "shucked_weight": np.float64,
    "viscera_weight": np.float64,
    "shell_weight": np.float64,
}
label_column_dtype = {"rings": np.float64}


def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z


if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    df = pd.read_csv(
        f"{base_dir}/input/abalone-dataset.csv",
        header=None,
        names=feature_columns_names + [label_column],
        dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype),
    )
    numeric_features = list(feature_columns_names)
    numeric_features.remove("sex")
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    )

    categorical_features = ["sex"]
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    y = df.pop("rings")
    X_pre = preprocess.fit_transform(df)
    y_pre = y.to_numpy().reshape(len(y), 1)

    X = np.concatenate((y_pre, X_pre), axis=1)

    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(0.7 * len(X)), int(0.85 * len(X))])

    pd.DataFrame(train).to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        f"{base_dir}/validation/validation.csv", header=False, index=False
    )
    pd.DataFrame(test).to_csv(f"{base_dir}/test/test.csv", header=False, index=False)

Overwriting abalone/preprocessing.py


다음으로 `SKLearnProcessor` 프로세서의 인스턴스를 만들고 `ProcessingStep`에서 사용합니다.

또한 이 노트북 전체에서 사용할 'framework_version'을 지정합니다.

프로세서 인스턴스에서 사용하는 'processing_instance_type' 및 'processing_instance_count' 매개변수에 유의하세요.

In [98]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    role=role,
)

마지막으로 프로세서 인스턴스를 사용하여 입력 및 출력 채널, 파이프라인이 파이프라인 실행을 호출할 때 실행되는 코드와 함께 `ProcessingStep`을 구성합니다. 이는 Python SDK에서 프로세서 인스턴스의 'run()' 메서드와 유사합니다.

`ProcessingStep`에 전달된 `input_data` 매개변수는 단계에서 사용되는 입력 데이터입니다. 이 입력 데이터는 프로세서 인스턴스가 실행될 때 사용됩니다.

또한 처리 작업에 대한 출력 구성에 지정된 `"train_data"` 및 `"test_data"` 이름의 채널에 유의하십시오. '속성' 단계는 후속 단계에서 사용할 수 있으며 실행 시 런타임 값으로 확인할 수 있습니다. 특히, 이 사용법은 훈련 단계를 정의할 때 호출됩니다.

In [99]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


step_process = ProcessingStep(
    name="AbaloneProcess",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", 
                         source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", 
                         source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", 
                         source="/opt/ml/processing/test"),
    ],
    code="abalone/preprocessing.py",
    cache_config = cache_config, # 캐시 정의
)

![Define a Processing Step for Feature Engineering](img/pipeline-2.png)

## Define a Training Step to Train a Model

이 섹션에서는 Amazon SageMaker의 [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)을 사용하여 이 데이터 세트를 교육합니다. XGBoost 알고리즘 및 입력 데이터 세트에 대한 Estimator를 구성합니다. 일반적인 훈련 스크립트는 입력 채널에서 데이터를 로드하고, 하이퍼파라미터로 훈련을 구성하고, 모델을 훈련하고, 나중에 호스팅할 수 있도록 모델을 `model_dir`에 저장합니다.

훈련의 모델이 저장되는 모델 경로도 지정됩니다.

'instance_type' 매개변수는 파이프라인의 여러 위치에서 사용될 수 있습니다. 이 경우 'instance_type'이 estimator로 전달됩니다.

In [100]:
from sagemaker.estimator import Estimator

model_path = f"s3://{default_bucket}/AbaloneTrain"

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=instance_type,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    disable_profiler=True   
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
)

마지막으로 estimator 인스턴스를 사용하여 `TrainingStep` 입력과 파이프라인이 파이프라인 실행을 호출할 때 실행되는 코드에서 입력으로 사용된 이전 `ProcessingStep`의 `properties`와 `TrainingStep`을 구성합니다. 이것은 Python SDK의 estimator의 'fit' 방법과 유사합니다.

`"train_data"` 출력 채널의 `S3Uri`를 `TrainingStep`에 전달합니다. 또한 파이프라인에서 모델 평가를 위해 다른 `"test_data"` 출력 채널을 사용합니다. 파이프라인 단계의 'properties' 속성은 설명 호출에 대한 해당 응답의 개체 모델과 일치합니다. 이러한 속성은 자리 표시자 값으로 참조될 수 있으며 런타임에 확인됩니다. 예를 들어 `ProcessingStep` `properties` 속성은 [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) 응답 객체의 객체 모델과 일치합니다.

In [101]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="AbaloneTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
    cache_config = cache_config, # 캐시 정의
)

![Define a Training Step to Train a Model](img/pipeline-3.png)

## Define a Model Evaluation Step to Evaluate the Trained Model

먼저 모델 평가를 수행하는 처리 단계에서 지정된 평가 스크립트를 개발합니다.

파이프라인 실행 후 분석을 위해 결과 'evaluation.json'을 검사할 수 있습니다.

평가 스크립트는 `xgboost`를 사용하여 다음을 수행합니다.

* 모델을 로드합니다.
* 테스트 데이터를 읽습니다.
* 테스트 데이터에 대한 예측을 발행합니다.
* 정확도 및 ROC 곡선을 포함하는 분류 보고서를 작성합니다.
* 평가 보고서를 평가 디렉토리에 저장합니다.

In [102]:
%%writefile abalone/evaluation.py
import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import mean_squared_error


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")

    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)

    X_test = xgboost.DMatrix(df.values)

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting abalone/evaluation.py


다음으로 `ScriptProcessor` 프로세서의 인스턴스를 만들고 `ProcessingStep`에서 사용합니다.

프로세서에 전달된 `processing_instance_type` 매개변수에 유의하십시오.

In [103]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-abalone-eval",
    role=role,
)

프로세서 인스턴스를 사용하여 입력 및 출력 채널과 파이프라인이 파이프라인 실행을 호출할 때 실행되는 코드와 함께 `ProcessingStep`을 구성합니다. 이는 Python SDK에서 프로세서 인스턴스의 'run' 메서드와 유사합니다.

특히, `step_train` `properties`의 `S3ModelArtifacts`와 `step_process` `properties`의 `"test_data"` output channel의 `S3Uri`이 입력으로 넘어갑니다. `TrainingStep`과 `ProcessingStep` `properties` 어트리뷰트는 각각 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html)의 object model과 [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects로 매칭됩니다.

In [104]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="AbaloneEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="abalone/evaluation.py",
    cache_config = cache_config, # 캐시 정의
    property_files=[evaluation_report],
)

![Define a Model Evaluation Step to Evaluate the Trained Model](img/pipeline-4.png)

## Define a Create Model Step to Create a Model

예제 모델을 사용하여 일괄 변환을 수행하려면 SageMaker 모델을 생성하십시오.

특히 `TrainingStep`, `step_train` 속성에서 `S3ModelArtifacts`를 전달합니다. `TrainingStep` `properties` 속성은 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) 응답 객체의 객체 모델과 일치합니다.

In [105]:
from sagemaker.model import Model


model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

SageMaker 모델 생성을 위한 모델 입력(`instance_type` 및 `accelerator_type`)을 제공한 다음 이전에 정의된 입력 및 모델 인스턴스를 전달하여 `CreateModelStep`을 정의합니다.

In [106]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.eia1.medium",
)
step_create_model = CreateModelStep(
    name="AbaloneCreateModel",
    model=model,
    inputs=inputs,
)

## Define a Transform Step to Perform Batch Transformation

이제 모델 인스턴스가 정의되었으므로 적절한 모델 유형, 컴퓨팅 인스턴스 유형 및 원하는 출력 S3 URI로 `Transformer` 인스턴스를 생성합니다.

특히 `CreateModelStep`, `step_create_model` 속성에서 `ModelName`을 전달합니다. `CreateModelStep` `properties` 속성은 [DescribeModel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html) 응답 객체의 객체 모델과 일치합니다.

In [107]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f"s3://{default_bucket}/AbaloneTransform",
)

이전에 정의한 `batch_data` 파이프라인 매개변수를 사용하여 transformer 인스턴스와 `TransformInput`을 전달합니다.

In [120]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="AbaloneTransform", 
    transformer=transformer, 
    inputs=TransformInput(data=batch_data),
    cache_config = cache_config # 캐시 정의
)

## Define a Register Model Step to Create a Model Package

훈련 단계에서 지정된 estimator 인스턴스를 사용하여 `RegisterModel`의 인스턴스를 구성합니다. 파이프라인에서 'RegisterModel'을 실행한 결과는 모델 패키지입니다. 모델 패키지는 추론에 필요한 모든 구성 요소를 패키징하는 재사용 가능한 모델 아티팩트의 추상화입니다. 기본적으로 optional 모델 가중치 위치와 함께 사용할 추론 이미지를 정의하는 추론 사양으로 구성됩니다.

모델 패키지 그룹은 모델 패키지의 모음입니다. 특정 ML 비즈니스 문제에 대해 모델 패키지 그룹을 생성할 수 있으며 여기에 모델 패키지의 새 버전을 추가할 수 있습니다. 일반적으로 고객은 모든 SageMaker 파이프라인 실행에 대해 모델 패키지 버전을 그룹에 추가할 수 있도록 SageMaker 파이프라인용 ModelPackageGroup을 생성해야 합니다.

'RegisterModel'의 구성은 Python SDK의 estimator 인스턴스의 'register' 메서드와 유사합니다.

특히 `TrainingStep`, `step_train` 속성에서 `S3ModelArtifacts`를 전달합니다. `TrainingStep` `properties` 속성은 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) 응답 객체의 객체 모델과 일치합니다.

이 노트북에 제공된 특정 모델 패키지 그룹 이름은 SageMaker 프로젝트와 함께 모델 레지스트리 및 CI/CD 작업에서 사용할 수 있습니다.

In [121]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)
step_register = RegisterModel(
    name="AbaloneRegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

![Define a Create Model Step and Batch Transform to Process Data in Batch at Scale](img/pipeline-5.png)

## Define a Fail Step to Terminate the Pipeline Execution and Mark it as Failed

이 섹션에서는 다음 단계를 안내합니다:

* 실행 실패의 원인을 나타내는 사용자 정의 오류 메시지와 함께 'FailStep'을 정의합니다.
* 보다 유익한 오류 메시지를 작성하기 위해 동적 `mse_threshold` 매개변수가 있는 정적 텍스트 문자열을 추가하는 `Join` 기능으로 `FailStep` 오류 메시지를 입력합니다.

In [122]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="AbaloneMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold]),
)

![Define a Fail Step to Terminate the Execution in Failed State](img/pipeline-8.png)

## Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry, Or Terminate the Execution in Failed State

이 단계에서는 평가 단계 'step_eval'에 의해 결정된 모델의 정확도가 지정된 값을 초과한 경우에만 모델을 등록합니다. 그렇지 않으면 파이프라인 실행이 실패하고 종료됩니다. 'ConditionStep'을 사용하면 파이프라인이 단계 속성의 조건에 따라 파이프라인 DAG에서 조건부 실행을 지원할 수 있습니다.

다음 섹션에서는 다음을 수행합니다:

* 평가 단계 'step_eval'의 출력에서 ​​찾은 정확도 값에 'ConditionLessThanOrEqualTo'를 정의합니다.
* `ConditionStep`의 조건 목록에 있는 조건을 사용합니다.
* `CreateModelStep` 및 `TransformStep` 단계와 `RegisterModel` 단계 컬렉션을 `ConditionStep`의 `if_steps`에 전달합니다. 이는 조건이 `True`로 평가되는 경우에만 실행됩니다.
* 조건이 'False'로 평가되는 경우에만 실행되는 'ConditionStep'의 'else_steps'에 'FailStep' 단계를 전달합니다.

In [123]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=mse_threshold,
)

step_cond = ConditionStep(
    name="AbaloneMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[step_fail],
)

![Define a Condition Step to Check Accuracy and Conditionally Execute Steps](img/pipeline-6.png)

## Define a Pipeline of Parameters, Steps, and Conditions

이 섹션에서는 실행할 수 있도록 단계를 파이프라인으로 결합합니다.

파이프라인에는 '이름', '매개변수', '단계'가 필요합니다. 이름은 `(account, region)` 쌍 내에서 고유해야 합니다.

메모:

* 정의에 사용된 모든 매개변수가 있어야 합니다.
* 파이프라인으로 전달된 단계는 실행 순서대로 나열할 필요가 없습니다. SageMaker 파이프라인 서비스는 실행을 완료하기 위한 단계로 데이터 종속성 DAG를 확인합니다.
* 단계는 파이프라인 단계 목록과 모든 조건 단계 if/else 목록에서 고유해야 합니다.

In [124]:
from sagemaker.workflow.pipeline import Pipeline


pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        mse_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

![Define a Pipeline of Parameters, Steps, and Conditions](img/pipeline-7.png)

### (Optional) Examining the pipeline definition

파이프라인 정의의 JSON을 검사하여 파이프라인이 잘 정의되어 있고 매개변수와 단계 속성이 올바르게 해석되는지 확인할 수 있습니다.

In [125]:
import json


definition = json.loads(pipeline.definition())
definition

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-387402383014/abalone/abalone-dataset.csv'},
  {'Name': 'BatchData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-387402383014/abalone/abalone-dataset-batch'},
  {'Name': 'MseThreshold', 'Type': 'Float', 'DefaultValue': 6.0}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'AbaloneProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingRes

## Submit the pipeline to SageMaker and start execution

파이프라인 정의를 파이프라인 서비스에 제출하십시오. 파이프라인 서비스는 전달된 역할을 사용하여 단계에서 정의된 모든 작업을 생성합니다.

In [126]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:pipeline/nickname-pipeline',
 'ResponseMetadata': {'RequestId': '0da6e8cd-1c06-4a53-aefe-306c55caaaa2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0da6e8cd-1c06-4a53-aefe-306c55caaaa2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Mon, 18 Apr 2022 14:12:41 GMT'},
  'RetryAttempts': 0}}

파이프라인을 시작하고 모든 기본 매개변수를 accept합니다.

In [127]:
execution = pipeline.start()

## Pipeline Operations: Examining and Waiting for Pipeline Execution

파이프라인 실행을 describe 합니다.

In [128]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:pipeline/nickname-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:pipeline/nickname-pipeline/execution/xw60js12v8jc',
 'PipelineExecutionDisplayName': 'execution-1650291162557',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'nickname-pipeline',
  'TrialName': 'xw60js12v8jc'},
 'CreationTime': datetime.datetime(2022, 4, 18, 14, 12, 42, 460000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 4, 18, 14, 12, 42, 460000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'aa33b260-664e-4dab-a483-c0ff8a3a3558',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aa33b260-664e-4dab-a483-c0ff8a3a3558',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '500',
   'date': 'Mon, 18 Apr 2022 14:12:44 GMT'},
  'RetryAttempts': 0}}

실행이 완료될 때까지 기다리십시오.

In [129]:
execution.wait()

실행 단계를 나열하십시오. 단계 실행기 서비스에서 해결한 파이프라인의 단계입니다.

In [130]:
execution.list_steps()

[{'StepName': 'AbaloneTransform',
  'StartTime': datetime.datetime(2022, 4, 18, 14, 12, 51, 812000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 4, 18, 14, 17, 22, 662000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:transform-job/pipelines-xw60js12v8jc-abalonetransform-iaagaapfb7'}}},
 {'StepName': 'AbaloneCreateModel',
  'StartTime': datetime.datetime(2022, 4, 18, 14, 12, 49, 9000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 4, 18, 14, 12, 50, 748000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:model/pipelines-xw60js12v8jc-abalonecreatemodel-iwhmakd0ug'}}},
 {'StepName': 'AbaloneRegisterModel',
  'StartTime': datetime.datetime(2022, 4, 18, 14, 12, 49, 9000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 4, 18, 14, 12, 49, 726000, tzinfo=tzloc

### Examining the Evaluation

파이프라인이 완료된 후 결과 모델 평가를 검토합니다. S3에서 결과 `evaluation.json` 파일을 다운로드하고 보고서를 인쇄합니다.

In [44]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mse': {'standard_deviation': 2.046578064221743,
                                'value': 4.188484906645659}}}


### Lineage

파이프라인에서 생성된 아티팩트의 lineage를 검토합니다.

In [46]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'AbaloneProcess', 'StartTime': datetime.datetime(2022, 4, 18, 7, 13, 29, 317000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 4, 18, 7, 18, 7, 588000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:processing-job/pipelines-1ouf8h3j3vip-abaloneprocess-bfgebyll50'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...1dc1ff1d9bf9/input/code/preprocessing.py,Input,DataSet,ContributedTo,artifact
1,s3://...387402383014/abalone/abalone-dataset.csv,Input,DataSet,ContributedTo,artifact
2,36674...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...50a0c87783b887401dc1ff1d9bf9/output/test,Output,DataSet,Produced,artifact
4,s3://...7783b887401dc1ff1d9bf9/output/validation,Output,DataSet,Produced,artifact
5,s3://...0a0c87783b887401dc1ff1d9bf9/output/train,Output,DataSet,Produced,artifact


{'StepName': 'AbaloneTrain', 'StartTime': datetime.datetime(2022, 4, 18, 7, 18, 8, 293000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 4, 18, 7, 21, 29, 489000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:training-job/pipelines-1ouf8h3j3vip-abalonetrain-omdecyltbl'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...7783b887401dc1ff1d9bf9/output/validation,Input,DataSet,ContributedTo,artifact
1,s3://...0a0c87783b887401dc1ff1d9bf9/output/train,Input,DataSet,ContributedTo,artifact
2,36674...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...loneTrain-OMdEcYlTbL/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'AbaloneEval', 'StartTime': datetime.datetime(2022, 4, 18, 7, 21, 30, 325000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 4, 18, 7, 26, 4, 577000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:processing-job/pipelines-1ouf8h3j3vip-abaloneeval-ctgzyy7p2o'}}}


...

,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...loneTrain-OMdEcYlTbL/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,36674...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,nickname-packagegroupname-1-PendingManualAppro...,Input,Approval,ContributedTo,action
3,NickName-PackageGroupName-1650266766-aws-model...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'AbaloneCreateModel', 'StartTime': datetime.datetime(2022, 4, 18, 7, 26, 6, 133000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 4, 18, 7, 26, 7, 238000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:model/pipelines-1ouf8h3j3vip-abalonecreatemodel-jyegg2kksf'}}}


None

{'StepName': 'AbaloneTransform', 'StartTime': datetime.datetime(2022, 4, 18, 7, 26, 7, 676000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 4, 18, 7, 30, 29, 603000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:387402383014:transform-job/pipelines-1ouf8h3j3vip-abalonetransform-ewgmkpjrf2'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...loneTrain-OMdEcYlTbL/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,36674...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,s3://...7402383014/abalone/abalone-dataset-batch,Input,DataSet,ContributedTo,artifact
3,s3://...ortheast-2-387402383014/AbaloneTransform,Output,DataSet,Produced,artifact


### Parametrized Executions

파이프라인의 추가 실행을 실행하고 다른 파이프라인 매개변수를 지정할 수 있습니다. `parameters` 인수는 매개변수 이름을 포함하는 사전이며, 기본값을 재정의하는 데 값이 사용됩니다.

모델의 성능에 따라 컴퓨팅 최적화 인스턴스 유형에서 다른 파이프라인 실행을 시작하고 모델 승인 상태를 자동으로 "승인됨"으로 설정할 수 있습니다. 이는 'RegisterModel' 단계에서 생성된 모델 패키지 버전이 SageMaker 프로젝트와 같은 CI/CD 파이프라인을 통해 배포할 수 있도록 자동 준비됨을 의미합니다.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.m5.large",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

그 외에도 MSE 임계값을 더 작은 값으로 조정하고 등록된 모델의 정확도에 대한 기준을 높일 수도 있습니다. 이 경우 다음과 같이 MSE 임계값을 무시할 수 있습니다:

In [ ]:
execution = pipeline.start(parameters=dict(MseThreshold=3.0))

MSE 임계값이 충족되지 않으면 파이프라인 실행이 `FailStep`에 들어가고 실패한 것으로 표시됩니다.

In [ ]:
try:
    execution.wait()
except Exception as error:
    print(error)

In [ ]:
execution.list_steps()